In [ ]:
import numpy as np
import xarray as xr
from kalman_reconstruction.pipeline import (
    xarray_Kalman_smoother_time_dependent,
    xarray_Kalman_SEM_time_dependent,
    add_random_variable,
    from_standard_dataset,
    to_standard_dataset,
    _input_matrices_H_R_from_n_p,
    forcast_from_kalman,
    perfect_forcast,
)
from kalman_reconstruction import example_models
from kalman_reconstruction.statistics import (
    coverage,
    xarray_coverage_prob,
    xarray_RMSE,
    normalize,
)
from kalman_reconstruction.custom_plot import (
    plot_state_with_probability,
    set_custom_rcParams,
    plot_colors,
    adjust_lightness,
)
import matplotlib.pyplot as plt
from kalman_reconstruction import statistics

from matplotlib.figure import Figure
from os import PathLike
from pathlib import Path
from typing import Dict
import seaborn as sns

set_custom_rcParams()
# plt.rcParams["figure.figsize"] = (8, 5)
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
## LIGHT THEME
plt.style.use("seaborn-v0_8-whitegrid")
dark_color = [0.3, 0.3, 0.3]
light_color = [0.8, 0.8, 0.8]
lightness_0 = 0.75
lightness_1 = 0.5
cmap = "rocket"
cmap_r = "rocket_r"

# ### DARK THEME
# plt.style.use("dark_background")
# dark_color = [0.7, 0.7, 0.7]
# light_color = [0.2, 0.2, 0.2]
# lightness_0 = 1.15
# lightness_1 = 1.5
# cmap = "rocket_r"
# cmap_r = "rocket"


colors = set_custom_rcParams()
plt.rcParams["axes.grid"] = True
plot_colors(colors)

variables_color = dict()
variables_color["x2"] = colors[0]
variables_color["x3"] = colors[1]
variables_color["z1"] = colors[2]
variables_color["z2"] = colors[3]
variables_color["z3"] = colors[4]
variables_color["x1"] = colors[5]

In [ ]:
REPO_PATH = Path(".").resolve().parent
results_path = REPO_PATH / Path("results") / "Report" / "method_enhancement"
results_path.mkdir(parents=True, exist_ok=True)
SAVE_FIGURES = True


def save_fig(fig: Figure, relative_path: PathLike, kwargs: Dict = dict()):
    store_path = results_path / relative_path
    store_path.parent.mkdir(parents=True, exist_ok=True)
    if SAVE_FIGURES:
        fig.savefig(store_path, **kwargs)
    else:
        pass

In [ ]:
seed = 39264
variance = 5
nb_iter_SEM = 30
dt = 0.005
number_loops = 10
forcast_duration = 0.5
forecast_length = int(forcast_duration / dt)

simga_in_time = 0.1
sigma = int(simga_in_time / dt)
print(sigma)

start_times = np.arange(10 * dt, number_loops, 2.6 * forcast_duration)
model_result = example_models.Lorenz_63_xarray(
    dt=dt, time_length=number_loops, time_steps=None
)

rng1 = np.random.default_rng(seed=seed)
rng2 = np.random.default_rng(seed=seed + 1)
rng3 = np.random.default_rng(seed=seed + 2)
rng4 = np.random.default_rng(seed=seed + 3)

20


### Plot example of Lorenz Model


In [ ]:
gs_kw = dict(height_ratios=[1.5, 1], hspace=0.001)
fig, axs = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(4, 5),
    sharex=False,
    sharey=False,
    gridspec_kw=gs_kw,
    layout="constrained",
)

stop = 5
for var in model_result.data_vars:
    axs[0].plot(
        model_result.time,
        model_result[var],
        label=var,
        linewidth=3,
    )

axs[0].set_xlabel("Time in cycles of Lorenz63")
axs[0].set_ylabel("Values of states")
# axs[0].set_title(r"Example Evolution of $\overline{x}$")
axs[0].legend(loc="lower right", handlelength=1)
axs[0].set_xlim(0, stop)
used_loops = model_result.sel(time=slice(0, stop))
sc = axs[1].scatter(
    x=used_loops["x2"],
    y=used_loops["x3"],
    c=used_loops["x1"],
    cmap="flare",
    s=5,
)
cbar = fig.colorbar(mappable=sc, ax=axs[1], location="right", pad=-0.04)
cbar.ax.set_title("x1", fontdict=dict(size=12))
axs[1].set_xlabel("x2")
axs[1].set_ylabel("x3")
# axs[1].set_title("Spacial Evolution")
# fig.tight_layout()
save_fig(fig, relative_path="Example_Lorenz63.pdf")

In [ ]:
# dictonaries to store the input data and the results of the Kalman-SEM
data_train = dict()
kalman_train = dict()

In [ ]:
names = [
    "0 latent",
    "1 latent",
    "2 latent",
    # "$x = [x_2, x_3, z_1, z_2, z_3]$",
]

### Apply the Kalman SEM using latent variables 


#### 0 latent variables

In [ ]:
new_name = names[0]

data_train[new_name] = model_result.copy()
# Run Klman_SEM
kalman_train[new_name] = xarray_Kalman_SEM_time_dependent(
    ds=data_train[new_name],
    observation_variables=["x2", "x3"],
    state_variables=["x2", "x3"],
    nb_iter_SEM=nb_iter_SEM,
    variance_obs_comp=0.0001,
    sigma=sigma,
)

fig, ax = plt.subplots(1, 1)
for var in kalman_train[new_name].state_name:
    var = str(var.values)
    plot_state_with_probability(
        ax=ax,
        x_value=kalman_train[new_name].time,
        state=kalman_train[new_name].states.sel(state_name=var),
        prob=kalman_train[new_name].covariance.sel(state_name=var, state_name_copy=var),
        line_kwargs=dict(label=var, color=variables_color[var]),
    )

ax.legend()
# ax.set_xlim((0, 2))
ax.set_xlabel("time")
ax.set_ylabel("Values")
ax.set_title("Using multiple random latent variables");

100%|██████████| 30/30 [34:21<00:00, 68.70s/it]


#### 1 latent variables

In [ ]:
old_name = names[0]
new_name = names[1]

data_train[new_name] = data_train[old_name].copy()
# Add latent variable
add_random_variable(
    data_train[new_name], var_name="z1", random_generator=rng1, variance=variance
)
# Run Klman_SEM
kalman_train[new_name] = xarray_Kalman_SEM_time_dependent(
    ds=data_train[new_name],
    observation_variables=["x2", "x3"],
    state_variables=["x2", "x3", "z1"],
    nb_iter_SEM=nb_iter_SEM,
    variance_obs_comp=0.0001,
    sigma=sigma,
)

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 6), sharex=True, sharey=True)

for var in data_train[new_name].data_vars:
    var = str(var)
    plot_state_with_probability(
        ax=axs[0],
        x_value=data_train[new_name].time,
        state=data_train[new_name][var],
        prob=0,
        line_kwargs=dict(label=var, color=variables_color[var]),
    )
for var in kalman_train[new_name].state_name:
    var = str(var.values)
    plot_state_with_probability(
        ax=axs[1],
        x_value=kalman_train[new_name].time,
        state=kalman_train[new_name].states.sel(state_name=var),
        prob=kalman_train[new_name].covariance.sel(state_name=var, state_name_copy=var),
        line_kwargs=dict(label=var, color=variables_color[var]),
    )


for ax in axs.flatten():
    ax.legend()
    # ax.set_xlim((0, 2))
    ax.set_xlabel("time")
    ax.set_ylabel("Values")
fig.suptitle("Using multiple random latent variables")

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [45:56<00:00, 91.88s/it] 


Text(0.5, 0.98, 'Using multiple random latent variables')

#### 2 latent variables

In [ ]:
old_name = names[1]
new_name = names[2]

data_train[new_name] = data_train[old_name].copy()
data_train[new_name]["z1"] = normalize(
    kalman_train[old_name].states.sel(state_name="z1")
)
# Add latent variable
add_random_variable(
    data_train[new_name], var_name="z2", random_generator=rng1, variance=variance
)
# Run Klman_SEM
kalman_train[new_name] = xarray_Kalman_SEM_time_dependent(
    ds=data_train[new_name],
    observation_variables=["x2", "x3"],
    state_variables=["x2", "x3", "z1", "z2"],
    nb_iter_SEM=nb_iter_SEM,
    variance_obs_comp=0.0001,
    sigma=sigma,
)

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 6), sharex=True, sharey=True)

for var in data_train[new_name].data_vars:
    var = str(var)
    plot_state_with_probability(
        ax=axs[0],
        x_value=data_train[new_name].time,
        state=data_train[new_name][var],
        prob=0,
        line_kwargs=dict(label=var, color=variables_color[var]),
    )
for var in kalman_train[new_name].state_name:
    var = str(var.values)
    plot_state_with_probability(
        ax=axs[1],
        x_value=kalman_train[new_name].time,
        state=kalman_train[new_name].states.sel(state_name=var),
        prob=kalman_train[new_name].covariance.sel(state_name=var, state_name_copy=var),
        line_kwargs=dict(label=var, color=variables_color[var]),
    )


for ax in axs.flatten():
    ax.legend()
    # ax.set_xlim((0, 2))
    ax.set_xlabel("time")
    ax.set_ylabel("Values")
fig.suptitle("Using multiple random latent variables")

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [25:11<00:00, 50.38s/it]


Text(0.5, 0.98, 'Using multiple random latent variables')

#### 3 latent variables

In [ ]:
# old_name = names[2]
# new_name = names[3]

# data_train[new_name] = data_train[old_name].copy()
# data_train[new_name]["z2"] = kalman_train[old_name].states.sel(state_name="z2")
# # Add latent variable
# add_random_variable(
#     data_train[new_name], var_name="z3", random_generator=rng1, variance=variance
# )
# # Run Klman_SEM
# kalman_train[new_name] = xarray_Kalman_SEM_time_dependent(
#     ds=data_train[new_name],
#     observation_variables=["x2", "x3"],
#     state_variables=["x2", "x3", "z1", "z2", "z3"],
#     nb_iter_SEM=nb_iter_SEM,
#     variance_obs_comp=0.0001, sigma = sigma,
# )

# fig, axs = plt.subplots(ncols = 2, nrows = 1, figsize = (12,6), sharex=True, sharey=True)

# for var in data_train[new_name].data_vars:
#     var = str(var)
#     plot_state_with_probability(
#         ax=axs[0],
#         x_value=data_train[new_name].time,
#         state=data_train[new_name][var],
#         prob=0,
#         line_kwargs=dict(label = var, color= variables_color[var]),
#     )
# for var in kalman_train[new_name].state_name:
#     var = str(var.values)
#     plot_state_with_probability(
#         ax=axs[1],
#         x_value=kalman_train[new_name].time,
#         state=kalman_train[new_name].states.sel(state_name=var),
#         prob=kalman_train[new_name].covariance.sel(state_name=var, state_name_copy=var),
#         line_kwargs=dict(label = var, color= variables_color[var]),
#     )


# for ax in axs.flatten():
#     ax.legend()
#     # ax.set_xlim((0, 2))
#     ax.set_xlabel("time")
#     ax.set_ylabel("Values")
# fig.suptitle("Using multiple random latent variables")

## Test the forecast skill
To test the forecast skill, we will create a test dataset starting from the end of the initial training data_train.

In [ ]:
# test_initial_condition = np.array(
#     [model_result[var].isel(time=-1).values for var in ["x1", "x2", "x3"]]
# )
# test_initial_condition

In [ ]:
# model_result_test = example_models.Lorenz_63_xarray(
#     dt=dt,
#     time_length=number_loops,
#     time_steps=None,
#     initial_condition=test_initial_condition,
# )
model_result_test = model_result.copy()

In [ ]:
data_test = dict()
kalman_test = kalman_train  # use the trained ``M`` and ``Q`` for the smoother later on
smoother_test = dict()
forecast_test = dict()

Create the test dataset with newly initialized latent varibables

Make sure to swap the random number generator between here and the initialization before to check if it still works fine.

It is sufficient here to only apply the Kalman smoother to the dataset in order to get proper results, as ``M`` and ``Q`` were trained on the period before.

In [ ]:
"""0 Latent"""

new_name = names[0]
data_test[new_name] = model_result_test.copy()

"""1 Latent"""
old_name = names[0]
new_name = names[1]
# copy dataset
data_test[new_name] = data_test[old_name].copy()
# Add latent variable
add_random_variable(
    data_test[new_name], var_name="z1", random_generator=rng3, variance=variance
)

"""2 Latent"""
old_name = names[1]
new_name = names[2]
# copy dataset
data_test[new_name] = data_test[old_name].copy()
data_test[new_name]["z1"] = data_test[old_name]["z1"]
# Add latent variable
add_random_variable(
    data_test[new_name], var_name="z2", random_generator=rng4, variance=variance
)


# # """3 Latent"""
# # old_name = names[2]
# # new_name = names[3]
# # # copy dataset
# # data_test[new_name] = data_test[old_name].copy()
# # data_test[new_name]["z2"] = data_test[old_name].states.sel(state_name="z2")
# # # Add latent variable
# # add_random_variable(
# #     data_test[new_name], var_name="z3", random_generator=rng1, variance=variance
# # )

In [ ]:
plt.plot(data_test[names[2]].x2, label="x2")
plt.plot(data_test[names[2]].x3, label="x3")
plt.plot(data_test[names[2]].z1, label="z1")
plt.plot(data_test[names[2]].z2, label="z2", zorder=0)

### Perform Kalman smoother using the last timestep as start.
The ``M`` and ``Q`` Matrices from the ``Kalman_SEM`` are used.

In [ ]:
idx = -1
"""0 Latent"""
observation_variables = ["x2", "x3"]
state_variables = ["x2", "x3"]
H, R = _input_matrices_H_R_from_n_p(
    n=len(state_variables), p=len(observation_variables)
)
smoother_test[names[0]] = xarray_Kalman_smoother_time_dependent(
    ds=data_test[names[0]],
    state_variables=state_variables,
    observation_variables=observation_variables,
    initial_covariance_matrix=kalman_test[names[0]].covariance.isel(time=idx),
    M=kalman_test[names[0]].M.values,
    Q=kalman_test[names[0]].Q.values,
    H=H,
    R=R,
    estimation_idx=idx,
    dim="time",
)

"""1 Latent"""
observation_variables = ["x2", "x3"]
state_variables = ["x2", "x3", "z1"]
H, R = _input_matrices_H_R_from_n_p(
    n=len(state_variables), p=len(observation_variables)
)
smoother_test[names[1]] = xarray_Kalman_smoother_time_dependent(
    ds=data_test[names[1]],
    state_variables=state_variables,
    observation_variables=observation_variables,
    initial_covariance_matrix=kalman_test[names[1]].covariance.isel(time=idx),
    M=kalman_test[names[1]].M.values,
    Q=kalman_test[names[1]].Q.values,
    H=H,
    R=R,
    estimation_idx=idx,
    dim="time",
)

"""2 Latent"""
observation_variables = ["x2", "x3"]
state_variables = ["x2", "x3", "z1", "z2"]
H, R = _input_matrices_H_R_from_n_p(
    n=len(state_variables), p=len(observation_variables)
)
smoother_test[names[2]] = xarray_Kalman_smoother_time_dependent(
    ds=data_test[names[2]],
    state_variables=state_variables,
    observation_variables=observation_variables,
    initial_covariance_matrix=kalman_test[names[2]].covariance.isel(time=idx),
    M=kalman_test[names[2]].M.values,
    Q=kalman_test[names[2]].Q.values,
    H=H,
    R=R,
    estimation_idx=idx,
    dim="time",
)

'''
# """3 Latent"""
# observation_variables = ["x2", "x3"]
# state_variables=["x2", "x3", "z1", "z2", "z3"]

# H, R = _input_matrices_H_R_from_n_p(n=len(state_variables), p=len(observation_variables))
# smoother_test[names[3]] = xarray_Kalman_smoother_time_dependent(
#     ds=data_test[names[2]],
#     state_variables=state_variables,
#     observation_variables=observation_variables,
#     initial_covariance_matrix=kalman_test[names[2]].covariance.isel(time=idx),
#     M=kalman_test[names[2]].M.values,
#     Q=kalman_test[names[2]].Q.values,
#     H=H,
#     R=R,
#     estimation_idx=idx,
#     dim="time",
# )
''';

In [ ]:
plt.plot(
    smoother_test[names[2]].state_smooth,
    label=smoother_test[names[2]].state_name.values,
)
plt.legend()

### Forcast of the system 
used to compute the RMSE and coverage_prbability

##### check if the Matrices are ordered correct
This can be used, but shows that a tranpose of the matrix ``M`` leads to bad results

In [ ]:
# kalman_test_false = dict()
# for key in names:
#     kalman_test_false[key] = kalman_test[key].copy()
#     da = kalman_test_false[key]["M"]
#     kalman_test_false[key]["M"] = (
#         ["time", "state_name", "state_name_copy"],
#         np.transpose(da.values, axes = [0,2,1])
#         )

In [ ]:
forecast_test = dict()
for nb_latent in names:
    forecast_test[nb_latent] = forcast_from_kalman(
        ds_kalman_SEM=kalman_test[nb_latent],
        ds_state_covariance=smoother_test[nb_latent],
        state_var_name="state_smooth",
        covariance_var_name="covariance_smooth",
        forecast_length=forecast_length,
    )

### Convert to more readable dataset

In [ ]:
forecast_test_state = dict()
forecast_test_covariance = dict()
for nb_latent in names:
    forecast_test_state[nb_latent] = from_standard_dataset(forecast_test[nb_latent])
    forecast_test_covariance[nb_latent] = from_standard_dataset(
        forecast_test[nb_latent], var_name="covariance"
    )


forecast_test_covariance[nb_latent]

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=False, figsize=(8, 5))

# plot x2
for var, ax in zip(["x2", "x3"], axs):
    ax.plot(
        model_result_test.time,
        model_result_test[var],
        color=dark_color,
        alpha=0.75,
        linewidth=2,
        # label=var,
    )
    ax.set_title(f"{var}")
for var, ax in zip(["x2", "x3"], axs):
    for idx, key in enumerate(names):
        state = forecast_test_state[key][var]
        prob = forecast_test_covariance[key][var].sel(state_name_copy=var)
        for jdx, start_time in enumerate(start_times):
            if jdx == 0:
                line_kwargs = dict(
                    color=colors[idx],
                    label=key,
                    linewidth=2,
                )
            else:
                line_kwargs = dict(
                    color=colors[idx],
                    linewidth=2,
                )
            plot_state_with_probability(
                ax=ax,
                x_value=start_time + state["horizon"] * dt,
                state=state.sel(time=start_time, method="nearest"),
                prob=prob.sel(time=start_time, method="nearest"),
                line_kwargs=line_kwargs,
                stds=0.64,
            )

axs[0].set_ylim([-30, 30])
ymin, ymax = axs[0].get_ylim()
axs[0].set_yticks(np.arange(ymin, ymax, 10))

axs[1].set_ylim([0, 45])
ymin, ymax = axs[1].get_ylim()
axs[1].set_yticks(np.arange(ymin, ymax, 10))

for ax in axs:
    ax.set_ylabel("Values")
    ax.legend()
axs[1].set_xlabel("Time")

Text(0.5, 0, 'Time')

### Create perfect forecast and compare to actuall forecast

In [ ]:
model_standard = to_standard_dataset(ds=model_result_test)
perfect = perfect_forcast(model_standard, forecast_length=forecast_length)
perfect = from_standard_dataset(perfect)

In [ ]:
rmse = dict()
cov_prob = dict()
for key in names:
    rmse[key] = xarray_RMSE(
        x=forecast_test_state[key],
        y=perfect,
        dim="time",
    )
    cov_prob[key] = xarray_coverage_prob(
        x=forecast_test_state[key],
        y=perfect,
        P=forecast_test_covariance[key],
        dim="time",
    )

c:\Software\Miniconda3\envs\climNum\lib\site-packages\xarray\core\computation.py:771: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
c:\Software\Miniconda3\envs\climNum\lib\site-packages\xarray\core\computation.py:771: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
c:\Software\Miniconda3\envs\climNum\lib\site-packages\xarray\core\computation.py:771: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [ ]:
states_to_plot = ["x2", "x3"]
fig = plt.figure()
subfigs = subfigs = fig.subfigures(ncols=1, nrows=2)

axs_rmse = subfigs[0].subplots(ncols=2, nrows=1, sharex=True, sharey=True)
axs_covp = subfigs[1].subplots(ncols=2, nrows=1, sharex=True, sharey=True)


for key in names:
    for idx, var in enumerate(["x2", "x3"]):
        # plot rmse:
        axs_rmse[idx].plot(perfect.horizon * dt, rmse[key][var], label=key)
        axs_rmse[idx].set_title(var)
        # plot rmse:
        axs_covp[idx].plot(
            perfect.horizon * dt, cov_prob[key][var].sel(state_name_copy=var), label=key
        )
        axs_covp[idx].set_xlabel("forecast horizon")
axs_rmse[0].set_ylabel("RMSE")
axs_covp[0].set_ylabel("coverage probability")

for ax in axs_rmse:
    ax.legend()
    ax.set_ylim((-1, 35))

for ax in axs_covp:
    ax.axhline(0.5, color=dark_color, alpha=0.5, linestyle=":")
    ax.legend()


# Write a function that takes in the dict of dataset

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4.5))

ax.plot(
    model_result_test["x2"],
    model_result_test["x3"],
    color=dark_color,
    alpha=0.75,
    label="truth",
)


for (key, ds), color in zip(
    forecast_test_state.items(), colors[0 : len(forecast_test)]
):
    for idx, start_time in enumerate(start_times):
        if idx == 0:
            line_kwargs = dict(color=color, label=key)
        else:
            line_kwargs = dict(color=color)
        ax.plot(
            ds["x2"].sel(time=start_time, method="nearest"),
            ds["x3"].sel(time=start_time, method="nearest"),
            **line_kwargs,
        )

ax.set_aspect("equal", adjustable="box")


ax.set_xlim([-30, 30])
xmin, xmax = ax.get_xlim()
ax.set_xticks(np.arange(xmin, xmax, 10), minor=True)

ymin, ymax = ax.set_ylim([0, 45])
ymin, ymax = ax.get_ylim()
ax.set_yticks(np.arange(ymin, ymax, 10))

ax.legend()

### Visualize the Kernel


In [ ]:
from kalman_reconstruction.statistics import gaussian_kernel_1D

fig, axs = plt.subplots(
    nrows=2, ncols=1, figsize=(4, 5), sharex=True, sharey=True, layout="constrained"
)
axs = axs.flatten()
kernel1 = gaussian_kernel_1D(
    x=model_result_test["x2"],
    center_idx=300,
    axis=0,
    sigma=sigma,
    same_output_shape=True,
)
kernel2 = gaussian_kernel_1D(
    x=model_result_test["x2"],
    center_idx=360,
    axis=0,
    sigma=sigma,
    same_output_shape=True,
)
for idx, k in enumerate([kernel1, kernel2]):
    sc = axs[idx].scatter(
        x=model_result_test["x2"],
        y=model_result_test["x3"],
        c=normalize(k, "minmax"),
        cmap=cmap,
        s=(normalize(k, "01") * 50) + 1,
        # alpha = 0.5,
    )
for ax in axs:
    ax.set_xlabel(r"$x_2$")
    ax.set_ylabel(r"$x_3$")
    ax.set_aspect("equal", adjustable="box")

fig.suptitle("Example kernels")
plt.colorbar(mappable=sc, ax=axs, label="LLR kernel values normalized")
save_fig(fig, relative_path="example_kernels.pdf")

In [ ]:
import seaborn as sns

fig, axs = plt.subplots(
    ncols=3, nrows=len(start_times), figsize=(10, 10), layout="constrained"
)
for idx, (key, ds) in enumerate(forecast_test.items()):
    ax = axs[:, idx]
    for j, start_time in enumerate(start_times):
        sns.heatmap(
            ds.M.sel(time=start_time, method="nearest"),
            ax=ax[j],
            cmap="RdBu_r",
            vmin=-1,
            vmax=1,
            square=True,
            annot=True,
        )
        ax[j].set_title(key + f"t={start_time:.2f}")

In [ ]:
from pathlib import Path

repo_path = Path(".").resolve().parent
dir_path = repo_path / Path(
    "data/processed/forecast_evaluation_high_resolution/time_dependent"
)
dir_path.mkdir(parents=True, exist_ok=True)

# save Input
save_path = f"input.nc"
model_result_test.to_netcdf(dir_path / save_path)

for key in names:
    save_name = key
    save_name = save_name.replace("$", "")
    save_name = save_name.replace(",", "")
    save_name = save_name.replace("[", "")
    save_name = save_name.replace("]", "")
    save_name = save_name.replace("_", "")
    save_name = save_name.replace(" ", "_")

    # save State
    save_path = f"state_{save_name}.nc"
    forecast_test_state[key].to_netcdf(dir_path / save_path)
    # save Covarinace
    save_path = f"covariance_{save_name}.nc"
    forecast_test_covariance[key].to_netcdf(dir_path / save_path)
    # save Coverage probability
    save_path = f"coverage_probability_{save_name}.nc"
    cov_prob[key].to_netcdf(dir_path / save_path)
    # save RMSE
    save_path = f"rmse_{save_name}.nc"
    rmse[key].to_netcdf(dir_path / save_path)